In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore')

from sklearn.model_selection import train_test_split

In [2]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
data = pd.read_csv(url, delimiter=';')
df = data.copy()
df.shape

(1599, 12)

# SMOTE

# Train/Test Split

# Procédure d'évaluation

# Modélisation

# Optimisation

# Bilan